In [11]:
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
import datetime
from vars import *

In [12]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator


dataset_path = dataset_path

log_dir = "../logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(150, 150),
    batch_size=75)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(150, 150),
    batch_size=75)


AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)


# train_datagen = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     validation_split=0.2) 


# train_generator = train_datagen.flow_from_directory(
#     dataset_path,
#     target_size=(150, 150),
#     batch_size=64,
#     class_mode='categorical', 
#     subset='training') 


# validation_generator = train_datagen.flow_from_directory(
#     dataset_path,
#     target_size=(150, 150),
#     batch_size=32,
#     class_mode='categorical',
#     subset='validation')

Found 9152 files belonging to 100 classes.
Using 7322 files for training.
Found 9152 files belonging to 100 classes.
Using 1830 files for validation.


In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomZoom, RandomCrop

model = Sequential([
    RandomFlip("horizontal", input_shape=(150, 150, 3)),
    RandomRotation(0.1),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(classes, activation='softmax')
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip_1 (RandomFlip)  (None, 150, 150, 3)       0         
                                                                 
 random_rotation_1 (RandomRo  (None, 150, 150, 3)      0         
 tation)                                                         
                                                                 
 conv2d_5 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 36, 36, 64)      

In [14]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=validation_dataset)

Epoch 1/10
98/98 [==============================] - 98s 541ms/step - loss: 5.7022 - accuracy: 0.0087 - val_loss: 4.6059 - val_accuracy: 0.0066
Epoch 2/10
98/98 [==============================] - 42s 430ms/step - loss: 4.6053 - accuracy: 0.0085 - val_loss: 4.6067 - val_accuracy: 0.0033
Epoch 3/10
98/98 [==============================] - 43s 437ms/step - loss: 4.6051 - accuracy: 0.0105 - val_loss: 4.6075 - val_accuracy: 0.0033
Epoch 4/10
98/98 [==============================] - 42s 430ms/step - loss: 4.6050 - accuracy: 0.0117 - val_loss: 4.6082 - val_accuracy: 0.0033
Epoch 5/10
98/98 [==============================] - 41s 418ms/step - loss: 4.6048 - accuracy: 0.0117 - val_loss: 4.6088 - val_accuracy: 0.0033
Epoch 6/10
98/98 [==============================] - 42s 433ms/step - loss: 4.6047 - accuracy: 0.0117 - val_loss: 4.6094 - val_accuracy: 0.0033
Epoch 7/10
98/98 [==============================] - 41s 423ms/step - loss: 4.6046 - accuracy: 0.0117 - val_loss: 4.6100 - val_accuracy: 0.0033